In [ ]:
import pandas as pd
import os
from math import floor, ceil
from pprint import pprint
from traceback import print_exc
import numpy as np


In [ ]:
import os

path = '/root/data/alok/slaughterreports'
pen_names = os.listdir(path)
print(pen_names[12:])

In [ ]:
# pen_names_new = [ pn.encode('utf-8', 'replace').decode() for pn in pen_names ]
# pen_names_new

In [ ]:
def clean_bounds(row):
    bounds = row['bounds']
    if row['category'] == 'Utkast' and bounds is np.nan:
        return (0,10)
    MISTYPES = {
        '[3. 4]': '[3, 4]',
        '[9+, ]': '[9, ]'
    }
    if bounds in MISTYPES:
        bounds = MISTYPES[bounds]
    if bounds is not np.nan:
        bounds = [x for x in bounds[1:-1].split(',') if x.strip()]
        bounds = [float(x) for x in bounds]
        if len(bounds) == 1:
            bounds = [bounds[0], 10]
        out = (floor(bounds[0]), min(ceil(bounds[1]), 10))
        return out
    else:
        return None
    
def clean_weight(w):
    if w is None:
        return w
    if isinstance(w, str):
        w = w.replace('..', '.').replace(' ', '')
        try:
            return float(w)
        except:
            print('Could not clean')
            print(w)
            return None
    elif isinstance(w, float):
        return w
    elif isinstance(w, int):
        return float(w)
    else:
        assert False

In [ ]:
f

In [ ]:
print('here')

out_d = dict()
for index, pen_f in enumerate(pen_names):
    print('\n\n\n\n\n\n')
    print(index, pen_f)

In [ ]:
# from traceback import print_exc/

In [ ]:
print('here')

out_d = dict()
for index, pen_name in enumerate(pen_names):
    print('\n\n\n\n\n\n')
    print(index, pen_f)
#     pen_name = pen_names_new[index]
    
    found = False
    this_pen_folder = os.path.join(path, pen_f)
    if not this_pen_folder.endswith('.DS_Store'):
        print('getting files...')
        files = os.listdir(this_pen_folder)
        for f in files:
            print('\n\n\n')
            print(pen_name, f)
            d = None
            if f.endswith('ods'):
                d = pd.read_excel(os.path.join(this_pen_folder, f), engine='odf')
                found = True
            elif f.endswith('csv'):
                print(pen_name)
                try:
                    d = pd.read_csv(os.path.join(this_pen_folder, f))
                    found = True
                except:
                    print('failed')
                    print(pen_name)
                    print(f)
                    continue
            else:
                continue

            d.columns = [col.lower().strip() for col in d.columns]

            d = d.rename({'catergory': 'category',
                          'caterory': 'category',
                         'net wett.': 'net weight',
                         'net wett': 'net weight',
                         'number of fish': 'number of fish'}, axis=1)
            required = [
                'Number of Fish',
                'category',
                'bounds',
                'net weight'
            ]
            if 'net weight' not in d.columns:
                d['net weight'] = None
            required = [r.lower() for r in required]
            has_required = True
            for col in required:
                if col not in d.columns:
                    has_required = False
                    print('missing', pen_name, col, d.columns)
            if not has_required:
                print(d)
                continue

            d['category'] = d['category'].apply(str)
            d['net weight'] = d['net weight'].apply(clean_weight)
            d['reg'] = d['category'].apply(lambda s: s[0].isdigit())
            d['bounds'] = d.apply(clean_bounds, axis=1)
            d.drop_duplicates(inplace=True)
            if d['reg'].sum() > 2:
                d = d[d['reg']]
            if len(d['category'].unique()) != len(d):
                d = d.groupby(['category', 'bounds'])[['number of fish', 'net weight']].sum().reset_index()

            if  not len(d['category'].unique()) == len(d):
                vcs = d['category'].value_counts()
                print('nonunique cats', pen_name, d)
                print(vcs[vcs>1])
            data = {
                 row['category']: (
                     row['bounds'],
                     row['number of fish'],
                     row['net weight']
                 ) for _, row in d.iterrows()
            }
            out_d[(pen_name, f)] = data
#             print((pen_name, f))
#             print(d)
        if not found:
            print('\n\nNo csv found for:')
            print(pen_name)
            print(files)

In [ ]:
print(len(out_d.keys()))

In [ ]:
set([k[0] for k in out_d])

In [ ]:
import pandas as pd

for k in out_d:
    print('\n\n\n\n')
    print(k)
    print(pd.DataFrame(out_d[k]).T)